In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
163,AFG,Asia,Afghanistan,2020-06-21,28424.0,546.0,569.0,21.0,730.162,14.026,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
268,ALB,Europe,Albania,2020-06-21,1891.0,53.0,43.0,1.0,657.099,18.417,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
437,DZA,Africa,Algeria,2020-06-21,11631.0,127.0,837.0,12.0,265.239,2.896,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-20,France,159969.0,0.0,29646.0,0.0,0.0,0.0
0,2020-06-20,China,84594.0,0.0,4650.0,0.0,0.0,0.0
0,2020-06-20,Italy,238477.0,0.0,34647.0,0.0,0.0,0.0
0,2020-06-20,Spain,246193.0,0.0,30231.0,0.0,0.0,0.0
0,2020-06-20,United States,2250047.0,0.0,119973.0,0.0,0.0,0.0
0,2020-06-20,World,8791549.0,0.0,468573.0,0.0,0.0,0.0
0,2020-06-20,United Kingdom,303400.0,0.0,42647.0,0.0,0.0,0.0
0,2020-06-20,Germany,189927.0,0.0,8908.0,0.0,0.0,0.0
0,2020-06-20,Iran,203119.0,0.0,9480.0,0.0,0.0,0.0
0,2020-06-20,Turkey,186467.0,0.0,4925.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-21,France,159969.0,160093.0,29646.0,29633.0,-124.0,13.0
0,2020-06-21,China,84594.0,84553.0,4650.0,4639.0,41.0,11.0
0,2020-06-21,Italy,238477.0,238275.0,34647.0,34610.0,202.0,37.0
0,2020-06-20,Spain,246193.0,245938.0,30231.0,30240.0,255.0,-9.0
0,2020-06-21,United States,2250047.0,2255119.0,119973.0,119719.0,-5072.0,254.0
0,2020-06-21,World,8791549.0,8795672.0,468573.0,466189.0,-4123.0,2384.0
0,2020-06-21,United Kingdom,303400.0,303110.0,42647.0,42589.0,290.0,58.0
0,2020-06-21,Germany,189927.0,189822.0,8908.0,8882.0,105.0,26.0
0,2020-06-21,Iran,203119.0,202584.0,9480.0,9507.0,535.0,-27.0
0,2020-06-21,Turkey,186467.0,186493.0,4925.0,4927.0,-26.0,-2.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")